# Pytorch Tutorial

### 1. Tensors and Dynamic Graphs

- Basic Operations involving ```torch.Tensor```.
- Introduction to the dynamic graphs of ```torch.Autograd```.

Setup torch and some variables

In [1]:
import torch

In [2]:
a = torch.ones(4,5) * 2
b = torch.ones(4,5) * 3

In [3]:
print('Tensor a:')
print(a)
print()
print('Tensor b:')
print(b)

Tensor a:
tensor([[2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.]])

Tensor b:
tensor([[3., 3., 3., 3., 3.],
        [3., 3., 3., 3., 3.],
        [3., 3., 3., 3., 3.],
        [3., 3., 3., 3., 3.]])


The most important feature of pytorch is its dynamic computational graph, which automatically computes the gradient with respect to tensors.
However not all tensors have their gradients calculated; only those with ```requires_grad = True``` do. Values of these tensors are cached during computation in the background.

```requires_grad``` is ```False``` by default when you create a tensor. This can be provided as an optional argument:
```python
a = torch.ones((4,5), requires_grad=True)
b = torch.ones((4,5), requires_grad=True)
```

In [4]:
print(a.requires_grad)
print(b.requires_grad)

False
False


In [5]:
a.requires_grad = True
b.requires_grad = True

In [6]:
gradient_tensor = a + b
print(gradient_tensor.requires_grad)

True


However, there are indeed situations where you do not want pytorch to keep track of the operations you make. ```with torch.no_grad():``` is for this.

In [7]:
with torch.no_grad():
    no_gradient_tensor = a + b

print(no_gradient_tensor.requires_grad)
print(no_gradient_tensor.data == gradient_tensor.data)

False
tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]], dtype=torch.uint8)


Let us perform simple operations to ```a``` and ```b```. The resulting ```m``` is a scalar value, represented in pytorch as a tensor with one element.

In [8]:
y = a + b
m = torch.mean(y)
print(m)

tensor(5., grad_fn=<MeanBackward1>)


The following implicitly calls ```m.backward(torch.Tensor([1]))```.

In [9]:
m.backward()

In [10]:
print(f'a.is_leaf={a.is_leaf}')
print('Gradient w.r.t tensor a:')
print(a.grad)
print()
print(f'b.is_leaf={b.is_leaf}')
print('Gradient w.r.t tensor b:')
print(b.grad)
print()
print(f'y.is_leaf={y.is_leaf}')
print('Gradient w.r.t tensor y:')
print(y.grad)

a.is_leaf=True
Gradient w.r.t tensor a:
tensor([[0.0500, 0.0500, 0.0500, 0.0500, 0.0500],
        [0.0500, 0.0500, 0.0500, 0.0500, 0.0500],
        [0.0500, 0.0500, 0.0500, 0.0500, 0.0500],
        [0.0500, 0.0500, 0.0500, 0.0500, 0.0500]])

b.is_leaf=True
Gradient w.r.t tensor b:
tensor([[0.0500, 0.0500, 0.0500, 0.0500, 0.0500],
        [0.0500, 0.0500, 0.0500, 0.0500, 0.0500],
        [0.0500, 0.0500, 0.0500, 0.0500, 0.0500],
        [0.0500, 0.0500, 0.0500, 0.0500, 0.0500]])

y.is_leaf=False
Gradient w.r.t tensor y:
None


New gradients generated by back propagation are added up.

In [11]:
m.backward()

In [12]:
print(f'a.is_leaf={a.is_leaf}')
print('Gradient w.r.t tensor a:')
print(a.grad)
print()
print(f'b.is_leaf={b.is_leaf}')
print('Gradient w.r.t tensor b:')
print(b.grad)

a.is_leaf=True
Gradient w.r.t tensor a:
tensor([[0.1000, 0.1000, 0.1000, 0.1000, 0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000]])

b.is_leaf=True
Gradient w.r.t tensor b:
tensor([[0.1000, 0.1000, 0.1000, 0.1000, 0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000]])


Reset variables.

In [17]:
a.grad = None
b.grad = None

The best part of dynamic computational graphs is that the graph is recreated from scratch at every iteration. Thus you don't have to encode all possible paths before you launch the training - what you run is what you differentiate.

[Autograd mechanics - Pytorch master documentation](https://pytorch.org/docs/stable/notes/autograd.html)

In [18]:
# torch.randn samples numbers from a standard normal distribution.
if torch.randn(1).item() > 0:
    print('y = torch.sum(a)')
    y = torch.sum(a)
else:
    print('y = torch.sum(b)')
    y = torch.sum(b)

y = torch.sum(a)


In [19]:
y.backward()

In [20]:
print(f'a.is_leaf={a.is_leaf}')
print('Gradient w.r.t tensor a:')
print(a.grad)
print()
print(f'b.is_leaf={b.is_leaf}')
print('Gradient w.r.t tensor b:')
print(b.grad)

a.is_leaf=True
Gradient w.r.t tensor a:
tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])

b.is_leaf=True
Gradient w.r.t tensor b:
None
